# Бета-банк проект

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. 

In [220]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

data = pd.read_csv('Churn.csv')

data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Признаки:

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак:
- Exited — факт ухода клиента

Преобразовать категориальные признаки в численные поможет техника прямого кодирования с учетом избытка данных, чтобы не попасть в ловушку фиктивных признаков.

In [221]:
data     = data.drop(['Surname','RowNumber','CustomerId'],axis=1)
data_ohe = pd.get_dummies(data, drop_first=True)
data_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Теперь мы разобьем датасет на обучающую, валидационную и тестовую выборки. Определимся с количественными признаками и стандартизируем их, чтобы алгоритм считал все признаки равнозначными.

In [222]:
#Извлекли целевой признак
target   = data_ohe['Exited']
features = data_ohe.drop(['Exited'] , axis=1)
#Заменим пропуски в столбце Tenure на медианное значение
features['Tenure'].fillna(features['Tenure'].median(), inplace=True)

#Разбили датасет в пропорциях 3:1:1
features_train, features_valid_test, target_train, target_valid_train = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid_test, target_valid_train, test_size=0.5, random_state=12345)

#Признаки, которые надо масштабировать
numeric = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

#Непосредственное масштабирование
scaler                  = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric]  = scaler.transform(features_test[numeric])

C:\Users\konst\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\konst\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\konst\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

Обучим модель без учета дисбаланса классов и затем сравним ее с константной моделью.

In [223]:
model           = DecisionTreeClassifier(random_state=12345,max_depth=8)
model.fit(features_train,target_train)
predicted_valid = model.predict(features_valid)
accuracy_valid  = accuracy_score(target_valid,predicted_valid)
print(accuracy_valid)

0.833


In [224]:
data_ohe['Exited'].value_counts(normalize=True)*100

0    79.63
1    20.37
Name: Exited, dtype: float64

In [225]:
target_pred_constant = pd.Series(0, index=target_valid.index)
accuracy = accuracy_score(target_valid,target_pred_constant)
print(accuracy)

0.7885


Как оказалось наше дерево решений оказалось не намного лучше константной модели. Все из-за дисбаланса классов. Количество отрицательных ответов составляет почти 80%.

Теперь для каждой модели будем пробовать разные методы борьбы с дисбалансом, при этом подбирая наилучшее значение гиперпараметра.

In [226]:
#Используем параметр class_weight
model = LogisticRegression(random_state=12345, solver='liblinear',class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)
print('AUC-ROC: ' + str(auc_roc))

F1: 0.4788245462402766
AUC-ROC: 0.7418085930882918


In [227]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [228]:
#Здесь опробуем upsampling
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)
model = LogisticRegression(solver='liblinear')
model.fit(features_upsampled,target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)
print('AUC-ROC: ' + str(auc_roc))

F1: 0.4834996162701458
AUC-ROC: 0.7420379539809105


In [229]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [230]:
#Здесь опробуем downsampling
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.24)

model = LogisticRegression(solver='liblinear')
model.fit(features_downsampled,target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)
print('AUC-ROC: ' + str(auc_roc))

F1: 0.48484848484848475
AUC-ROC: 0.7407577304364902


In [231]:
features_downsampled_valid, target_downsampled_valid = downsample(features_valid, target_valid, 0.24)

features_new_train = pd.concat([features_downsampled,features_downsampled_valid])
target_new_train   = pd.concat([target_downsampled,target_downsampled_valid])

model = LogisticRegression(solver='liblinear')
model.fit(features_new_train,target_new_train)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)
print('AUC-ROC: ' + str(auc_roc))

F1: 0.49462365591397844
AUC-ROC: 0.747145356341379


Для модели логистической регрессии были опробованы три метода борьбы с дисбалансом классов. Сильной разницы между значением качества не было обнаружено, но тем не менее, выбрали downsample, поскольку он дал слегка более высокое значение. В результате преобразовали валидационную выборку этим методом и смешали с обучающей выборкой, чтобы переобучить модель на большем датасете. Не смотря на это, результат оказался недостаточно высоким, поэтому отбрасываем данную модель.

Теперь проделаем тоже самое с деревом решений.

In [232]:
#Пробуем downsampling
for m_d in np.arange(4,16,2):
    model                   = DecisionTreeClassifier(random_state=12345, max_depth=m_d)
    model.fit(features_downsampled,target_downsampled)
    predictions             = model.predict(features_valid)
    f1_score_val            = round(f1_score(target_valid,predictions),4)
    probabilities_valid     = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(m_d) + '. F1_score = ' + str(f1_score_val) + '. AUC-ROC = ' + str(auc_roc))
    print()

Глубина дерева = 4. F1_score = 0.508. AUC-ROC = 0.8088

Глубина дерева = 6. F1_score = 0.5592. AUC-ROC = 0.8192

Глубина дерева = 8. F1_score = 0.5447. AUC-ROC = 0.7847

Глубина дерева = 10. F1_score = 0.5268. AUC-ROC = 0.7595

Глубина дерева = 12. F1_score = 0.504. AUC-ROC = 0.7219

Глубина дерева = 14. F1_score = 0.495. AUC-ROC = 0.7094



При downsampling наибольшее значение F1-меры получено при максимальной глубине дерева, равное 6.

In [233]:
for m_d in np.arange(4,16,2):
    model                   = DecisionTreeClassifier(random_state=12345, max_depth=m_d,class_weight='balanced')
    model.fit(features_train,target_train)
    predictions             = model.predict(features_valid)
    f1_score_val            = round(f1_score(target_valid,predictions),4)
    probabilities_valid     = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(m_d) + '. F1_score = ' + str(f1_score_val) + '. AUC-ROC = ' + str(auc_roc))
    print()

Глубина дерева = 4. F1_score = 0.5385. AUC-ROC = 0.8187

Глубина дерева = 6. F1_score = 0.5644. AUC-ROC = 0.8194

Глубина дерева = 8. F1_score = 0.5477. AUC-ROC = 0.7798

Глубина дерева = 10. F1_score = 0.518. AUC-ROC = 0.7428

Глубина дерева = 12. F1_score = 0.5131. AUC-ROC = 0.7148

Глубина дерева = 14. F1_score = 0.522. AUC-ROC = 0.7081



Тот же результат при использовании параметра class_weight.

In [234]:
for m_d in np.arange(4,16,2):
    model                    = DecisionTreeClassifier(random_state=12345, max_depth=m_d,class_weight='balanced')
    model.fit(features_upsampled,target_upsampled)
    predictions              = model.predict(features_valid)
    f1_score_val             = round(f1_score(target_valid,predictions),4)
    predictions_             = model.predict(features_train)
    f1_score_val_            = round(f1_score(target_train,predictions_),4)
    probabilities_valid      = model.predict_proba(features_valid)
    probabilities_one_valid  = probabilities_valid[:, 1]
    auc_roc                  = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(m_d) + '. F1_score_valid = ' + str(f1_score_val) + ' F1-score_train = ' + str(f1_score_val_) + '. AUC-ROC = ' + str(auc_roc))
    print()

Глубина дерева = 4. F1_score_valid = 0.5385 F1-score_train = 0.5175. AUC-ROC = 0.8187

Глубина дерева = 6. F1_score_valid = 0.5644 F1-score_train = 0.5835. AUC-ROC = 0.8194

Глубина дерева = 8. F1_score_valid = 0.5514 F1-score_train = 0.6557. AUC-ROC = 0.785

Глубина дерева = 10. F1_score_valid = 0.5257 F1-score_train = 0.7068. AUC-ROC = 0.7484

Глубина дерева = 12. F1_score_valid = 0.5211 F1-score_train = 0.8122. AUC-ROC = 0.7171

Глубина дерева = 14. F1_score_valid = 0.5124 F1-score_train = 0.9094. AUC-ROC = 0.6995



Кажется рассмотренные методы примерно одинаково влияют на качество обученной модели. В последнем цикле также было расчитано качество на обучающей выборке, чтобы проконтроллировать переобучение.

Теперь объединим обучающую и валидационную выборку, чтобы заново обучить модель с наилучшим значением гиперпараметра.

In [235]:
features_new_train      = pd.concat([features_train,features_valid])
target_new_train        = pd.concat([target_train,target_valid])

model                   = DecisionTreeClassifier(random_state=12345, max_depth=6,class_weight='balanced')
model.fit(features_new_train,target_new_train)
predictions             = model.predict(features_test)
f1_score_val            = round(f1_score(target_test,predictions),4)
probabilities_valid     = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc                 = round(roc_auc_score(target_test,probabilities_one_valid),4)
print('F1-мера = ' + str(f1_score_val))
print('AUC-ROC = ' + str(auc_roc))

F1-мера = 0.5714
AUC-ROC = 0.8343


Близко, но, к сожалению, придется отбросить и эту модель.

Переходим к случайному лесу.

In [236]:
#Пробуем downsampling
for depth in np.arange(4,20,2):
    model                   = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=12345)
    model.fit(features_downsampled, target_downsampled)
    predictions             = model.predict(features_valid)
    probabilities_valid     = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(depth) + '. F1_score = ' + str(round(f1_score(target_valid,predictions),4)) \
          + '. AUC-ROC = ' + str(auc_roc))

Глубина дерева = 4. F1_score = 0.5632. AUC-ROC = 0.8235
Глубина дерева = 6. F1_score = 0.5809. AUC-ROC = 0.8399
Глубина дерева = 8. F1_score = 0.5663. AUC-ROC = 0.8366
Глубина дерева = 10. F1_score = 0.5704. AUC-ROC = 0.8403
Глубина дерева = 12. F1_score = 0.5513. AUC-ROC = 0.8368
Глубина дерева = 14. F1_score = 0.5584. AUC-ROC = 0.8286
Глубина дерева = 16. F1_score = 0.563. AUC-ROC = 0.8276
Глубина дерева = 18. F1_score = 0.5634. AUC-ROC = 0.8257


Наилучшим получили качество при глубине дерева 6.

In [237]:
#Пробуем upsampling
for depth in np.arange(4,20,2):
    model                   = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=12345)
    model.fit(features_upsampled, target_upsampled)
    predictions             = model.predict(features_valid)
    probabilities_valid     = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(depth) + '. F1_score = ' + str(round(f1_score(target_valid,predictions),4)) \
          + '. AUC-ROC = ' + str(auc_roc))

Глубина дерева = 4. F1_score = 0.5739. AUC-ROC = 0.8339
Глубина дерева = 6. F1_score = 0.5874. AUC-ROC = 0.8432
Глубина дерева = 8. F1_score = 0.5869. AUC-ROC = 0.8455
Глубина дерева = 10. F1_score = 0.5973. AUC-ROC = 0.8479
Глубина дерева = 12. F1_score = 0.5951. AUC-ROC = 0.8469
Глубина дерева = 14. F1_score = 0.5904. AUC-ROC = 0.845
Глубина дерева = 16. F1_score = 0.535. AUC-ROC = 0.8297
Глубина дерева = 18. F1_score = 0.5882. AUC-ROC = 0.8359


Получили наибольшее качество при глубине дерева 10. При этмо качество увеличили почти на .02.

In [238]:
#Балансировка через параметр модели
for depth in np.arange(4,20,2):
    model                   = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=12345,class_weight='balanced')
    model.fit(features_train, target_train)
    predictions             = model.predict(features_valid)
    probabilities_valid     = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
    print('Глубина дерева = ' + str(depth) + '. F1_score = ' + str(round(f1_score(target_valid,predictions),4)) \
          + '. AUC-ROC = ' + str(auc_roc))

Глубина дерева = 4. F1_score = 0.5686. AUC-ROC = 0.8211
Глубина дерева = 6. F1_score = 0.5916. AUC-ROC = 0.842
Глубина дерева = 8. F1_score = 0.6015. AUC-ROC = 0.8483
Глубина дерева = 10. F1_score = 0.5885. AUC-ROC = 0.8471
Глубина дерева = 12. F1_score = 0.5595. AUC-ROC = 0.842
Глубина дерева = 14. F1_score = 0.5347. AUC-ROC = 0.8273
Глубина дерева = 16. F1_score = 0.5217. AUC-ROC = 0.8293
Глубина дерева = 18. F1_score = 0.5. AUC-ROC = 0.8389


В последнем блоке кода было получено максимальное качество 0.6 при глубине дерева 8. Остановимся на последнем методе и используем его вместе с полученным значением гиперпараметра для того, чтобы обучить случайный лес с большим количеством оценщиков (еще один гиперпараметр). 

In [239]:
model                   = RandomForestClassifier(n_estimators=105, max_depth=8, random_state=12345,class_weight='balanced')
model.fit(features_train, target_train)
predictions             = model.predict(features_valid)
print('F1-мера = ' + str(round(f1_score(target_valid,predictions),4)))
probabilities_valid     = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc                 = round(roc_auc_score(target_valid,probabilities_one_valid),4)
print('AUC-ROC = ' + str(auc_roc))

F1-мера = 0.6167
AUC-ROC = 0.8574


В принципе, мы получили необходимое значение F1-меры, но можно ее еще увеличить через слияние обучающей и валидационной выборок.

In [240]:
features_new_train      = pd.concat([features_train,features_valid])
target_new_train        = pd.concat([target_train,target_valid])
model                   = RandomForestClassifier(n_estimators=104, max_depth=8, random_state=12345,class_weight='balanced')
model.fit(features_new_train, target_new_train)
predictions             = model.predict(features_test)
print('F1-мера = ' + str(round(f1_score(target_test,predictions),4)))
probabilities_valid     = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc                 = round(roc_auc_score(target_test,probabilities_one_valid),4)
print('AUC-ROC = ' + str(auc_roc))

F1-мера = 0.6258
AUC-ROC = 0.858


## Вывод

В данном проекте столкнулись с задачей классификации, для которой опробовали три модели: логистическую регрессию, решающее дерево и случайный лес. Сначала обучили модель без учета дисбаланса классов и сравнили ее с константной моделью. Данный шаг показал, что если не учитывать дисбаланс, то точность модели не будет сильно отличаться от константной.

На данном этапе нам известно всего 3 способа борьбы с дисбалансом классов: взвешивание классов через параметр class_weight, upsampling и downsampling. В данном проекте, ради интереса, для каждой модели был использован каждый из перечисленных методов, чтобы посмотреть какой лучше подойдет и есть ли зависимость между результатами для одной и той же модели при различных методах борьбы с дисбалансом классов.

Далее были масштабированы численные переменные и начат анализ моделей. Сначала обучили и потестировали Логистическую регрессию, которая в самом лучшем случае давала недостаточно высокое значение F1-меры, поэтому ее сразу отбросили.

Следом за Логистической регрессией пошло Дерево решений, результаты которого не сильно менялись от метода балансировки. Определившись со значением гиперпараметра и обучив модель на выборке, состоявшей из обучающей и валидационной, мы получили более высокое значение качества, однако, все еще недостаточное, чтобы остановиться.

В конце концов, подошел Случайный лес. Определившись со значениями двух гиперпараметров, мы и без слияния двух выборок получили необходимое значение F1-меры, но тем не менее увеличили ее и этим до значения 0.6258. 

Стоит отметить, что метрика AUC-ROC в циклах, как и F1-мера, сначала росла с увеличением значения гиперпараметра, а затем падала. Ее максимальное значение также было на выбранных нами значениях гиперпараметров.